In [1]:
# imports
from qiskit.aqua import QuantumInstance
from qiskit import BasicAer
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE, IQPE
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import ExcitationPreserving

from qiskit.circuit.library import ExcitationPreserving

from qiskit.chemistry.core import ChemistryOperator

from qiskit import BasicAer
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.aqua.algorithms import VQAlgorithm, VQE, MinimumEigensolver
from qiskit.chemistry.qubit_transformations import FermionicTransformation
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.ground_state_calculation import MinimumEigensolverGroundStateCalculation

In [2]:
# import BOPES
from qiskit.chemistry.bopes_sampler import BOPESSampler

In [3]:
ft = FermionicTransformation()
driver = PySCFDriver()
solver = VQE(quantum_instance=
             QuantumInstance(backend=BasicAer.get_backend('statevector_simulator')))
#solver = NumPyMinimumEigensolver()
me_gsc = MinimumEigensolverGroundStateCalculation(ft, solver)

In [4]:
# import molecule 
from qiskit.chemistry.drivers.molecule import Molecule
from functools import partial

In [5]:
stretch1 = partial(Molecule.absolute_stretching, atom_pair=(1, 0))
mol = Molecule(geometry=[('H', [0., 0., 0.]),
                        ('H', [0., 0., 1.])],
                       degrees_of_freedom=[stretch1],
                       )

# pass molecule to PSYCF driver
driver = PySCFDriver(molecule=mol)

In [6]:
# BOPES sampler testing
bs = BOPESSampler(gsc=me_gsc, driver=driver, 
                  bootstrap=True)

bs_res = bs.run([0,1,2,3,4,5])

Initial params , point:  None 0
[]
Optimal Params, point:  [-1.12391558 -0.62909081  1.69390607  2.20470298  1.59865125  2.2031501
  0.70632102  1.7283395 ] 0
Initial params , point:  None 1
[0]
Optimal Params, point:  [-1.10378126 -0.44494107  1.94961058  2.57538256  1.57001688  1.98307187
  0.69697354  1.73121102] 1
Initial params , point:  [-1.12391558 -0.62909081  1.69390607  2.20470298  1.59865125  2.2031501
  0.70632102  1.7283395 ] 2
[0, 1]
Optimal Params, point:  [-1.05447419 -0.35869892  2.03844965  2.73274971  1.51696677  1.85549196
  0.69618112  1.7473939 ] 2
Initial params , point:  [-1.12391558 -0.62909081  1.69390607  2.20470298  1.59865125  2.2031501
  0.70632102  1.7283395 ] 3
[0, 1, 2]
Optimal Params, point:  [-1.05009804 -0.34410614  2.05842091  2.76278691  1.51202842  1.83577311
  0.69459317  1.74948524] 3
Initial params , point:  [-1.12391558 -0.62909081  1.69390607  2.20470298  1.59865125  2.2031501
  0.70632102  1.7283395 ] 4
[0, 1, 2, 3]
Optimal Params, point:  [